#  **ChatGPT-4o com RAG de arquivos e Embedding**

**Authors:** Renato Freitas Bessa, Adonias Caetano de Oliveira, Ariel Soares Teles

**Institution:** UFDPar - PPGBiotec

**Objective:** responde os exame de 2024.01 do revalida com GPT-4o por meio de RAG baseado em arquivos e usando embedding.

## **Installation**

In [1]:
!pip install pandas openai faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 16.8 MB/s eta 0:00:00


## **Importing libraries**

In [2]:
import os
import openai
import faiss
import pandas as pd
import re
import random
import os
import asyncio
from openai import AsyncOpenAI
import numpy as np

## **Loading data**

In [3]:
#sheet_names = ["2017.01", "2017.02", "2020.01", "2020.02", "2021.01", "2022.01", "2022.02", "2023.01", "2023.02", "2024.01"]
sheet_names = ["2024.01"]

In [ ]:
url = '<public link to access the questions from the 2017.01 to 2024.01 exams>'
file_id = url.split('/')[-2]
read_url='https://drive.google.com/uc?id=' + file_id

# read the data
provas = pd.read_excel(read_url)
provas.head()

,Perguntas-2017.01,Gabarito-2017.01,Perguntas-2017.02,Gabarito-2017.02,Perguntas-2020.01,Gabarito-2020.01,Perguntas-2020.02,Gabarito-2020.02,Perguntas-2021.01,Gabarito-2021.01,Perguntas-2022.01,Gabarito-2022.01,Perguntas-2022.02,Gabarito-2022.02,Perguntas-2023.01,Gabarito-2023.01,Perguntas-2023.02,Gabarito-2023.02,Perguntas-2024.01,Gabarito-2024.01
0,1.\tUma mulher com 23 anos de idade comparece ...,NULA,1.\tUma mulher com 38 anos de idade procurou a...,C,1.\tAo atender uma primigesta com 15 anos de i...,A,"1.\tUma mulher com 43 anos de idade, com 80 kg...",C,"1.\tPrimigesta, de 25 anos de idade, com 34 se...",B,"1.\tUma mulher com 51 anos de idade, em tratam...",NULA,1.\tIMAGEM,C,"1.\tUm paciente com 54 anos, em acompanhamento...",D,"1.\tUma mulher de 60 anos, hipertensa, em uso ...",B,"1 Uma mulher com 25 anos, solteira, sem comorb...",A
1,2.\tUm homem com 65 anos de idade encontra-se ...,B,2.\tUma mulher com 23 anos de idade comparece ...,NULA,2.\tUma mulher com 35 anos de idade exerce pro...,C,2.\tAo atender uma primigesta com 15 anos de i...,A,2.\tUm recém-nascido com 20 dias de vida dá en...,C,"2.\tUm paciente com 72 anos de idade, tabagist...",C,"2.\tHomem de 75 anos de idade, tabagista 1 maç...",A,"2.\tUm homem com 62 anos comparece, acompanhad...",D,2.\tUm paciente de 48 anos é internado devido ...,D,"2 Um paciente com 28 anos, vítima de atropelam...",A
2,3.\tUma mulher com 38 anos de idade procurou a...,C,3.\tUm homem com 65 anos de idade encontra-se ...,B,"3.\tUma mulher com 43 anos de idade, com 80 kg...",C,3.\tUma mulher com 35 anos de idade exerce pro...,C,"3.\t Um homem, de 37 anos de idade, com AIDS/H...",B,"3.\tUm menino com 5 anos de idade, acompanhado...",C,3.\tLactente de 6 meses está sendo atendido na...,B,"3.\tUma lactente com 6 meses, nascida a termo,...",B,"3.\tUm menino de 9 anos, com peso de 29 kg e a...",D,3 Um lactente com 3 meses comparece a uma cons...,C
3,4.\tIMAGEM,IMAGEM,"4.\tUma mulher com 32 anos de idade, no quinto...",B,4.\tUma mulher com 18 anos de idade vai ao amb...,A,4.\tUma mulher com 48 anos de idade vai a uma ...,D,4.\tPaciente teve parto cesárea há 8 dias e re...,D,4.\tSegundo o Instituto Nacional do Câncer (IN...,D,"4.\tAdolescente de 14 anos de idade, primípara...",D,"4.\tUma paciente com 21 anos, nuligesta, compa...",A,4.\tUma mulher de 49 anos comparece à unidade ...,B,"4 Uma gestante com 24 anos, G2 P1 A0, chega à ...",D
4,"5.\tUma criança com 4 anos de idade, cujos pai...",A,5.\tIMAGEM,B,5.\tUma criança com 6 anos de idade é levada p...,B,5.\tUma mulher com 18 anos de idade vai ao amb...,A,"5.\tPaciente de 68 anos de idade, do sexo masc...",B,"5.\tUm homem com 48 anos de idade é obeso, tab...",A,5.\tUma mulher de 34 anos com diagnóstico de d...,D,5.\tSindemia é um conjunto de problemas de saú...,A,5.\tEm um território com população em situação...,B,5 Ao assumir a coordenação de uma equipe de sa...,A


In [5]:
# Passo 1: Carrega a planilha
df = pd.read_excel("RAG.xlsx")
perguntas = df["perguntas"].astype(str).tolist()
respostas = df["respostas"].astype(str).tolist()

## **ChatGPT model**

In [ ]:
OPENAI_API_KEY="<api key>"

In [7]:
client = AsyncOpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

##  **Gera embeddings dos dados**

In [8]:
# Passo 2: Gera embeddings das perguntas
# Make the gerar_embeddings function asynchronous
async def gerar_embeddings(client, textos, modelo="text-embedding-3-small"):
    embeddings = []
    for i in range(0, len(textos), 100):  # Lote de até 100
        lote = textos[i:i+100]
        # Use the provided client to create embeddings and await the result
        resposta = await client.embeddings.create(input=lote, model=modelo)
        lote_embeddings = [d.embedding for d in resposta.data]
        embeddings.extend(lote_embeddings)
    return embeddings

# Since gerar_embeddings is now async, you need to await it
# This code block also needs to be within an async function or awaited directly
async def setup_embeddings():
    vetores = await gerar_embeddings(client, perguntas)
    vetores_np = np.array(vetores).astype("float32")

    # Passo 3: Cria índice FAISS
    dim = len(vetores[0])
    index = faiss.IndexFlatL2(dim)
    index.add(vetores_np)
    return index, vetores_np

# Await the setup function to initialize index and vetores_np
index, vetores_np = await setup_embeddings()

In [14]:
async def consulta_RAG(pergunta_usuario, k=3):
    # Embedding da pergunta do usuário
    # Pass the client and the list containing the user question
    query_embedding = await gerar_embeddings(client, [pergunta_usuario])
    # Access the embedding from the list returned by gerar_embeddings
    query_np = np.array(query_embedding).astype("float32")

    # Busca as perguntas mais semelhantes
    distancias, indices = index.search(query_np, k)

    # Monta contexto com perguntas e respostas mais próximas
    contexto = ""
    for idx in indices[0]:
        contexto += f"Pergunta: {perguntas[idx]}\nResposta: {respostas[idx]}\n\n"

    return contexto

## **Respondendo as questões do Revalida**

In [15]:
from google.colab import files

def save_respostas(respostas_llm, name):
  df = pd.DataFrame(respostas_llm)
  df.to_excel(f'respostas-{name}.xlsx', index=False)

  # Especificar o caminho do arquivo no Google Drive
  files.download(f'respostas-{name}.xlsx')

In [16]:
async def main() -> None:
  respostas_llm = {}

  for name in sheet_names:
    perguntas = provas['Perguntas-' + name].to_list()
    gabaritos = provas['Gabarito-' + name].to_list()
    respostas_llm['Perguntas-' + name] = perguntas
    respostas_llm['Gabarito-' + name] = gabaritos
    respostas_llm['Repostas-' + name] = []

    for pergunta, gabarito in zip(perguntas, gabaritos):

      if gabarito != "IMAGEM" and gabarito != "TABELA" and gabarito != "NULA":

        # Introduce a delay to avoid hitting the rate limit
        #await asyncio.sleep(20) # Adjust the sleep time as needed

        prompt = f"Dado a questão a seguir, qual item está correto? Responda com a letra do item correto: {pergunta}"

        # Await the call to the async consulta_RAG function
        contexto = await consulta_RAG(prompt)

        response = await client.chat.completions.create(
              model="gpt-4o",
              messages=[
                   {
                      "role": "system",
                      "content": "Use sua base de conhecimento e a Base RAG."
                  },
                  {
                      "role": "user",
                      "content": f"Base RAG:\n{contexto}"
                  },
                  {"role": "user",
                   "content": prompt}
              ]
        )
        # Access the content using dot notation instead of square brackets
        resposta = response.choices[0].message.content

        respostas_llm['Repostas-' + name].append(resposta.strip())
      else:
        respostas_llm['Repostas-' + name].append(gabarito)

    save_respostas(respostas_llm, name)

# Use await directly instead of asyncio.run()
await main()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>